# Historical Tweets - from Twitter Search API using Tweepy Library <br>

Heavily inspired by DSI-6's(Amy Taylor and Veronica Giannotta) work

In [124]:
import pandas as pd
import tweepy
import shutil
import jsonpickle
import json
import datetime
import config
from pandas.io.json import json_normalize

## Creation of API object


In [125]:
# authenticate account with tweepy
auth = tweepy.OAuthHandler('1ZWD1JS7xivKADeOCV7zVXgI4', 'TgsgO0MaUYw2dccUVC7KtD8SYUX4QYaoxT52D4ye2146d0sLHo')
auth.set_access_token('3799487133-rGUwhJRC2pKZmx8UbureNKPxXSdZAgsWnOrjMQk', 'jOr0vlL0gzyRh1gOQTievE2ZAvuN1sD2FgYCGilFKmE60')

# create API object to pull data from twitter - and pass in authentication code
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# City api.geo search

In [137]:
#  Determine the city, country, or location you want
places = api.geo_search(query="Boston, MA", 
                        granularity="city"
                        )
place_id = places[0].id
print('place_id is: ',place_id)

place_id is:  67b98f17fdcf20be


**Download tweets from Search API based on our query search terms, and save tweets as a json file**

In [138]:
# query including placeID
searchQuery = 'place:67b98f17fdcf20be closed traffic' 

tweetCount = 0

#Open a text file to save the tweets to
with open('./boston.json', 'w') as f:

    # use Search API (api.search) with query
    for tweet in tweepy.Cursor(api.search,q=searchQuery
                              ).items() :         

        #Verify the tweet has place info before writing 
        if tweet.place is not None:
            
            #Write the JSON format to the text file, and add one to the number collected
            f.write(jsonpickle.encode(tweet._json, unpicklable=False) + '\n')
            tweetCount += 1

    #Display number of tweets collected
    print("Downloaded {0} tweets".format(tweetCount))

Downloaded 12 tweets


**Create a timestamp to the json file name**

In [139]:
def file_conversion():
    #create a timestamp
    now = datetime.datetime.now()
    month = '0'+str(now.month)
    day = str(now.day)
    hour = str(now.hour)
    minute = str(now.minute)
    now_str = str(now.year)+month+day+'_'+hour+'_'+minute
    
    
    # Dali: I did not get this part of the code
    #replace the name of the file with a new timestamped filename
    
    dest = './boston_' + now_str + ".json"
    shutil.move('./boston.json', dest)
    
    with open(dest, "r") as f:
        status = f.readlines()
        jsons = []
        for ind in status:
            jsons.append(json.loads(ind))
    return jsons

# Uncomment out the file_conversion function to activate the file name change
len(file_conversion())

12

**Add the newly created json file to the list below for easy access later**

In [140]:
# Previously created json files can be accessed from this list
#json_df = pd.read_json("./medford_20190415_17_41.json", lines = True)
#json_df = pd.read_json("./medford_20190415_21_24.json", lines = True)
json_df = pd.read_json("./boston_20190415_22_44.json", lines = True)

**Explore the Tweets from json file**

In [141]:
json_df.shape

(12, 25)

In [142]:
json_df.head(2)

,contributors,coordinates,created_at,entities,favorite_count,favorited,geo,id,id_str,in_reply_to_screen_name,...,lang,metadata,place,possibly_sensitive,retweet_count,retweeted,source,text,truncated,user
0,NaN,"{'coordinates': [-71.07923, 42.34756], 'type':...",2019-04-15 19:03:54,"{'hashtags': [{'indices': [19, 26], 'text': 'B...",2,False,"{'coordinates': [42.34756, -71.07923], 'type':...",1117866152829448192,1117866152829448192,None,...,en,"{'iso_language_code': 'en', 'result_type': 're...","{'attributes': {}, 'bounding_box': {'coordinat...",0.0,0,False,"<a href=""http://www.sigalert.com/Map.asp?regio...",Off-ramp closed in #Boston on Mass Pike EB at ...,False,"{'contributors_enabled': False, 'created_at': ..."
1,NaN,"{'coordinates': [-71.07923, 42.34756], 'type':...",2019-04-15 13:41:53,"{'hashtags': [{'indices': [19, 26], 'text': 'B...",2,False,"{'coordinates': [42.34756, -71.07923], 'type':...",1117785115378241538,1117785115378241536,None,...,en,"{'iso_language_code': 'en', 'result_type': 're...","{'attributes': {}, 'bounding_box': {'coordinat...",NaN,0,False,"<a href=""http://www.sigalert.com/Map.asp?regio...",Off-ramp closed in #Boston on Mass Pike EB at ...,False,"{'contributors_enabled': False, 'created_at': ..."


In [143]:
# feature names
json_df.columns

Index(['contributors', 'coordinates', 'created_at', 'entities',
       'favorite_count', 'favorited', 'geo', 'id', 'id_str',
       'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'metadata',
       'place', 'possibly_sensitive', 'retweet_count', 'retweeted', 'source',
       'text', 'truncated', 'user'],
      dtype='object')

**View all tweets from the `text` column** (for the first 7 tweets)

**reduce down to only the useful columns**

In [144]:
boston = json_df[['coordinates', 'created_at', 'geo', 'place','text']]

In [145]:
boston

,coordinates,created_at,geo,place,text
0,"{'coordinates': [-71.07923, 42.34756], 'type':...",2019-04-15 19:03:54,"{'coordinates': [42.34756, -71.07923], 'type':...","{'attributes': {}, 'bounding_box': {'coordinat...",Off-ramp closed in #Boston on Mass Pike EB at ...
1,"{'coordinates': [-71.07923, 42.34756], 'type':...",2019-04-15 13:41:53,"{'coordinates': [42.34756, -71.07923], 'type':...","{'attributes': {}, 'bounding_box': {'coordinat...",Off-ramp closed in #Boston on Mass Pike EB at ...
2,"{'coordinates': [-71.07923, 42.34756], 'type':...",2019-04-15 13:09:16,"{'coordinates': [42.34756, -71.07923], 'type':...","{'attributes': {}, 'bounding_box': {'coordinat...",Off-ramp closed in #Boston on Mass Pike EB at ...
3,"{'coordinates': [-71.07923, 42.34756], 'type':...",2019-04-11 04:45:33,"{'coordinates': [42.34756, -71.07923], 'type':...","{'attributes': {}, 'bounding_box': {'coordinat...",Closed in #Boston on Mass Pike EB at IC 22 - C...
4,"{'coordinates': [-71.06112, 42.33037], 'type':...",2019-04-11 04:30:41,"{'coordinates': [42.33037, -71.06112], 'type':...","{'attributes': {}, 'bounding_box': {'coordinat...",Closed in #Boston on Rt 93 NB from EX 16 - Sou...
5,"{'coordinates': [-71.06112, 42.33037], 'type':...",2019-04-11 04:27:06,"{'coordinates': [42.33037, -71.06112], 'type':...","{'attributes': {}, 'bounding_box': {'coordinat...",Closed in #Boston on Rt 93 NB from EX 16 - Sou...
6,"{'coordinates': [-71.06112, 42.33037], 'type':...",2019-04-11 04:15:19,"{'coordinates': [42.33037, -71.06112], 'type':...","{'attributes': {}, 'bounding_box': {'coordinat...",Closed in #Boston on Rt 93 NB from EX 16 - Sou...
7,"{'coordinates': [-71.05365, 42.36284], 'type':...",2019-04-10 03:37:11,"{'coordinates': [42.36284, -71.05365], 'type':...","{'attributes': {}, 'bounding_box': {'coordinat...",Closed in #Boston on Rt 1A SB at Sumner Tunl #...
8,"{'coordinates': [-71.03821, 42.37323], 'type':...",2019-04-09 06:08:51,"{'coordinates': [42.37323, -71.03821], 'type':...","{'attributes': {}, 'bounding_box': {'coordinat...",Closed in #Boston on Rt 1A NB at Callahan Tunl...
9,None,2019-04-08 23:44:33,None,"{'attributes': {}, 'bounding_box': {'coordinat...",@shaleentitle Also pretty misleading photos on...


### Explore some features of the df

In [146]:
# see all the nested information within place (for the first two tweets)
list = boston.loc[:1, 'place']
for i in range(len(list)):
    print(i, list[i])
    print("--------")

0 {'attributes': {}, 'bounding_box': {'coordinates': [[[-71.191421, 42.227797], [-70.986004, 42.227797], [-70.986004, 42.399542], [-71.191421, 42.399542]]], 'type': 'Polygon'}, 'contained_within': [], 'country': 'United States', 'country_code': 'US', 'full_name': 'Boston, MA', 'id': '67b98f17fdcf20be', 'name': 'Boston', 'place_type': 'city', 'url': 'https://api.twitter.com/1.1/geo/id/67b98f17fdcf20be.json'}
--------
1 {'attributes': {}, 'bounding_box': {'coordinates': [[[-71.191421, 42.227797], [-70.986004, 42.227797], [-70.986004, 42.399542], [-71.191421, 42.399542]]], 'type': 'Polygon'}, 'contained_within': [], 'country': 'United States', 'country_code': 'US', 'full_name': 'Boston, MA', 'id': '67b98f17fdcf20be', 'name': 'Boston', 'place_type': 'city', 'url': 'https://api.twitter.com/1.1/geo/id/67b98f17fdcf20be.json'}
--------


> The dataframe that we have now has many attributes nested within other attributes. If we want to access the corrdinates nested within `coordinates` or the city nested within `place` we will need to unpack these variables using a different method. Use the next section to convert nested values into their own columns.

## Option B. Tweets --> json --> parsed --> nested dataframe

In [147]:
# Read in json file
boston = pd.read_json("./boston_20190415_22_44.json", lines = True)

# convert json file to list of dictionaries
tweets_data = []
notParsed = []
tweets_file = open("./boston_20190415_22_44.json","r")
for line in tweets_file:    
    if line.strip():    
        try:
            tweet=json.loads(line)
            tweets_data.append(tweet)
        except:
            notParsed.append(line)
            continue
print(len(tweets_data))
print('Could not parse: ', len(notParsed))

12
Could not parse:  0


**Unpack the nested columns**

In [148]:
tweet_cols = ['coordinates', 'created_at', 
              'text','geo', 'id', 'place', 'user']

sample_tweets_dict = [{col:tweet[col] for col in tweet_cols } for tweet in tweets_data]

# This method looks for any instances of nested dictionaries (DOES NOT FLATTEN LISTS)
sample_tweets_df = pd.io.json.json_normalize(sample_tweets_dict)

sample_tweets_df.head(2)

,coordinates,coordinates.coordinates,coordinates.type,created_at,geo,geo.coordinates,geo.type,id,place.bounding_box.coordinates,place.bounding_box.type,...,user.profile_text_color,user.profile_use_background_image,user.protected,user.screen_name,user.statuses_count,user.time_zone,user.translator_type,user.url,user.utc_offset,user.verified
0,NaN,"[-71.07923, 42.34756]",Point,Mon Apr 15 19:03:54 +0000 2019,NaN,"[42.34756, -71.07923]",Point,1117866152829448192,"[[[-71.191421, 42.227797], [-70.986004, 42.227...",Polygon,...,333333,True,False,TotalTrafficBOS,62031,None,none,None,None,False
1,NaN,"[-71.07923, 42.34756]",Point,Mon Apr 15 13:41:53 +0000 2019,NaN,"[42.34756, -71.07923]",Point,1117785115378241538,"[[[-71.191421, 42.227797], [-70.986004, 42.227...",Polygon,...,333333,True,False,TotalTrafficBOS,62031,None,none,None,None,False


In [149]:
sample_tweets_df.columns

Index(['coordinates', 'coordinates.coordinates', 'coordinates.type',
       'created_at', 'geo', 'geo.coordinates', 'geo.type', 'id',
       'place.bounding_box.coordinates', 'place.bounding_box.type',
       'place.contained_within', 'place.country', 'place.country_code',
       'place.full_name', 'place.id', 'place.name', 'place.place_type',
       'place.url', 'text', 'user.contributors_enabled', 'user.created_at',
       'user.default_profile', 'user.default_profile_image',
       'user.description', 'user.entities.description.urls',
       'user.entities.url.urls', 'user.favourites_count',
       'user.follow_request_sent', 'user.followers_count', 'user.following',
       'user.friends_count', 'user.geo_enabled', 'user.has_extended_profile',
       'user.id', 'user.id_str', 'user.is_translation_enabled',
       'user.is_translator', 'user.lang', 'user.listed_count', 'user.location',
       'user.name', 'user.notifications', 'user.profile_background_color',
       'user.profile_bac

In [150]:
# save the USA (or Bston) historical json file as df, COMMENT OUT
df = sample_tweets_df
df.to_csv("./boston.csv", index = False)

Examine who the most common tweets are coming from

In [151]:
sample_tweets_df['user.screen_name'].value_counts().head()

TotalTrafficBOS    10
berkie1             1
pmarvin123          1
Name: user.screen_name, dtype: int64

So the Twitter user "TotalTraffic", specific to a certain city, is contributing the most tweets to situations with "road closed", and providing location place data for each tweet